In [1]:
import numpy as np
import math
import pandas as pd
from hdmm import workload, fairtemplates, error, fairmechanism, matrix, mechanism, templates

# Understanding vectorized databases
This section is to help me better understand and visualize linear queries and query workloads as vectors and matrices. I'll be using the example of a toy dataset and a few queries in Figure 2.1 from the [Li, et. al 2014 paper](https://people.cs.umass.edu/~mcgregor/papers/15-vldbj.pdf) to familiarize myself with working with these objects using numpy.

Load in the toy dataset from the paper's diagram about students' information as `toy`:

In [2]:
toy = pd.read_csv('toydataset.csv')
toy

,Name,Gradyear,Gender,Gpa
0,Alice,2012,F,3.8
1,Bob,2011,M,3.1
2,Charlie,2014,M,3.6
3,Dave,2014,M,3.3
4,Evelyn,2013,F,3.9
5,Frank,2011,M,3.2
6,Gary,2015,M,3.5


You can represent the above `toy` database as a vector __x__, where $x_i$ represents the counts of $\phi_i$ for i from 1 through 8. There are 8 spots because there are 4 levels for Gradyear and 2 levels for Gender; 4 * 2 = 8. 

Let R = (name, gradyear, gender, gpa) be the schema for the database. Each $\phi_i$ represents the schema for all M or F in a specific year. For instance, $\phi_1$ represents all the males in 2011, i.e. R(\*, 2011, M, \*), and $\phi_2$ represents all the women in 2011, i.e. R(*, 2011, F, *). 

__x__ is the vector representation of the above database:

In [3]:
x_toy = np.array([2, 0, 0, 1, 0, 1, 2, 0])

Each index has the counts for each schema corresponding to it (i.e., the year and gender pairing). 

Here is a workload of 5 queries, where $w_i$ represent different queries, such as: 

- $w_1$: Students of any gender with gradyear ∈ [2011, 2014]
- $w_2$: Students with gradyear ∈ [2011, 2012]
- $w_3$: Female students with gradyear ∈ [2011, 2012]
- $w_4$: Male students with gradyear ∈ [2011, 2012]
- $w_5$: Difference between 2013 grads and 2014 grads

Combine these queries into a workload matrix called `W_toy`:

In [4]:
W_toy = np.array([[1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 0, 0, 0, 0],
         [0, 1, 0, 1, 0, 0, 0, 0],
         [1, 0, 1, 0, 0, 0, 0, 0], 
         [0, 0, 0, 0, 1, 1, -1, -1]])

Now, you can evaluate each query $w_i$ by dotting it with __x__, or by multiplying the workload matrix by __x__:

In [5]:
np.dot(W_toy[0], x_toy) # evaluating the first query in the workload

6

In [6]:
np.matmul(W_toy, x_toy) # evaluating the entire workload with x

array([ 6,  3,  1,  2, -1])

# Private Multiplicative Weights

In this section, I'll attempt to implement the private multiplicative weights algorithm from the [Hardt, et al 2014 paper](https://guyrothblum.files.wordpress.com/2014/11/hr10.pdf), and I will apply it to four examples. 

In [7]:
def pmw(W, x, eps = 0.01, beta = 0.1):
    '''
    Implement Private Multiplicative Weights Mechanism (PMW) on a workload of linear queries.
    
    - W = workload of queries (M x k numpy array)
    - x = true database (M x 1 numpy array)
    '''
    
    M = x.size # len of database
    n = x.sum() # sum of database
    k = len(W) # num of queries
    delta = 1/(n*math.log(n, 2))
    
    x_norm = x / np.sum(x)
    #print(f'x_norm: {x_norm}')
    eta = math.log(M, 2)**(1/4)/math.sqrt(n)
    sigma = 10 * math.log(1/delta, 2) * (math.log(M, 2))**(1/4)/(math.sqrt(n)*eps) 
    T = 4 * sigma * (math.log(k, 2) + math.log(1/beta, 2)) # threshold
    
    # initialize synthetic database at time 0 (prior to any queries)
    y_t = np.ones(M)/M
    x_t = np.ones(M)/M # fractional histogram computed in round t
    
    # append to list of databases y_t and x_t
    y_list = [y_t]
    x_list = [x_t]
    
    update_count = 0
    query_answers = []
    d_t_hat_list = []
    
    # iterate through time = [1, k] 
    for t in range(1, k + 1): 
        
        # compute noisy answer by adding Laplacian noise 
        A_t = np.random.laplace(0, sigma, 1)[0]  
        a_t_hat = np.dot(W[t - 1], x) + A_t
        #print(f'a_t_hat: {a_t_hat}')
        
        # compute difference between noisy answer and answer from maintained histogram
        d_t_hat = a_t_hat - np.dot(W[t - 1], x_list[t - 1]) 
        d_t_hat_list.append(d_t_hat)
        #print(f'd_{t}-hat (diff between noisy vs non-noisy answer for query t={t}): {d_t_hat}')
        
        # lazy round: use already maintained histogram to answer the query
        if (abs(d_t_hat) <= T): 
            query_answers.append(np.dot(W[t - 1], x_list[t - 1]))
            x_list.append(x_list[t - 1])
            continue
            
        # update round: update the histogram and return the noisy answer, abs(d_t_hat) > T
        else: 
            update_count+=1
            #step a
            r_t = np.zeros(M)
            if d_t_hat > 0:
                r_t = W[t - 1]
            else: 
                r_t = np.ones(M) - W[t - 1]
            for i in range(len(x_t)):
                y_t[i] = x_list[t - 1][i] * math.exp(-eta * W[t - 1][i])
            y_list.append(y_t)
            
            #step b
            x_t = y_t / np.sum(y_t)
            x_list.append(x_t)
            
        if update_count > n * math.log(M)**(1/2):
            return "failure"
        else: 
            query_answers.append(a_t_hat)
    
    d_t_hat_average = sum(d_t_hat_list) / len(d_t_hat_list)
    
    print(f'T (Threshold) = {T}')    
    print(f'query_answers (using pmw): {query_answers}')
    print(f'Update Count = {update_count}')  
    
    return query_answers, d_t_hat_average

# 5/30

### Ex. 1. pmw using workload from example from Li, et. al 2014 paper

In [8]:
pmw(W_toy, x_toy) # run pmw using the toy datasets/queries from above

T (Threshold) = 47973.204587234504
query_answers (using pmw): [1.0, 0.5, 0.25, 0.25, 0.0]
Update Count = 0


([1.0, 0.5, 0.25, 0.25, 0.0], -85.34213809657243)

### Ex. 2. pmw using Identity Total Workload

In [9]:
W1 = workload.IdentityTotal(8).dense_matrix()
pmw(W1, x_toy)

T (Threshold) = 55181.24244638598
query_answers (using pmw): [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 1.0]
Update Count = 0


([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 1.0],
 2329.583543624549)

In [10]:
W1

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.]])

### Ex. 3. pmw using All Range Workload

In [11]:
W2 = workload.AllRange(8).dense_matrix()
pmw(W2, x_toy)

T (Threshold) = 72181.39375220772
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125]
Update Count = 0


([0.125,
  0.25,
  0.375,
  0.5,
  0.625,
  0.75,
  0.875,
  1.0,
  0.125,
  0.25,
  0.375,
  0.5,
  0.625,
  0.75,
  0.875,
  0.125,
  0.25,
  0.375,
  0.5,
  0.625,
  0.75,
  0.125,
  0.25,
  0.375,
  0.5,
  0.625,
  0.125,
  0.25,
  0.375,
  0.5,
  0.125,
  0.25,
  0.375,
  0.125,
  0.25,
  0.125],
 -337.836600137244)

### Ex. 4. pmw using H2 Workload

In [12]:
W3 = workload.H2(8).dense_matrix()
pmw(W3, x_toy)

T (Threshold) = 61445.50575039114
query_answers (using pmw): [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 1.0]
Update Count = 0


([0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.25,
  0.25,
  0.25,
  0.25,
  0.5,
  0.5,
  1.0],
 -1572.1491298753401)

## Ex. 5. Males in one year only

In [13]:
x_ex_5 = np.array([1000, 1210, 0, 1250, 0, 1450, 0, 1720])
pmw(W_toy, x_ex_5)

T (Threshold) = 5969.915901974942
query_answers (using pmw): [6593.77724776444, 0.5, 0.25, 0.25, 0.0]
Update Count = 1


([6593.77724776444, 0.5, 0.25, 0.25, 0.0], 2716.178169032678)

# 6/22

## 1. Changing size of dataset to diagnose update issue

In [14]:
x_very_small = np.array([10, 12, 13, 12, 15, 14, 17, 17])
pmw(W_toy, x_very_small) 

T (Threshold) = 27033.472469855726
query_answers (using pmw): [1.0, 0.5, 0.25, 0.25, 0.0]
Update Count = 0


([1.0, 0.5, 0.25, 0.25, 0.0], 71.68911444181515)

In [15]:
x_small = np.array([100, 121, 130, 125, 150, 145, 170, 172])
pmw(W_toy, x_small) 

T (Threshold) = 11986.562604181201
query_answers (using pmw): [1.0, 0.5, 0.25, 0.25, 0.0]
Update Count = 0


([1.0, 0.5, 0.25, 0.25, 0.0], 438.3879663768039)

In [16]:
x_large = np.array([1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720])
pmw(W_toy, x_large) 

T (Threshold) = 4841.347513104868
query_answers (using pmw): [11160.584793559707, 0.5, 0.25, 0.25, 0.0]
Update Count = 1


([11160.584793559707, 0.5, 0.25, 0.25, 0.0], 4122.290956906191)

In [17]:
x_really_large = np.array([10000, 12100, 13000, 12500, 15000, 14500, 17000, 17200])
pmw(W_toy, x_really_large) 

T (Threshold) = 1855.1842600792784
query_answers (using pmw): [111563.89841393253, 47692.41029157478, 24579.48568816281, 23228.416756948674, -4931.80620907952]
Update Count = 5


([111563.89841393253,
  47692.41029157478,
  24579.48568816281,
  23228.416756948674,
  -4931.80620907952],
 40426.081136483124)

Making the dataset a lot larger didn't change much. 

In [18]:
x_really_large_variant = np.array([5000, 6100, 13000, 9500, 8100, 9000, 12000, 14000])
pmw(W_toy, x_really_large_variant) 

T (Threshold) = 2172.1248163621963
query_answers (using pmw): [76559.30590566077, 33593.820909276496, 15419.589511298891, 18015.74317218225, -9292.69942878613]
Update Count = 5


([76559.30590566077,
  33593.820909276496,
  15419.589511298891,
  18015.74317218225,
  -9292.69942878613],
 26858.75219248092)

Conclusion: Adding variation didn't really change much, x still isn't updating.

## 2. Adding fields to dataset to diagnose Update Issue

In [19]:
# 8 fields
pmw(W_toy, x_toy) 

T (Threshold) = 47973.204587234504
query_answers (using pmw): [1.0, 0.5, 0.25, 0.25, 0.0]
Update Count = 0


([1.0, 0.5, 0.25, 0.25, 0.0], -682.2561566946002)

In [20]:
# 32 fields
x_more_fields = np.array([1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720, 2150, 2300, 3100, 3200, 4000, 4300, 4334, 5000, 1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720, 2150, 2300, 3100, 3200, 4000, 4300, 4334, 5000, 1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720, 2150, 2300, 3100, 3200, 4000, 4300, 4334, 5000, 1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720, 2150, 2300, 3100, 3200, 4000, 4300, 4334, 5000, 1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720, 2150, 2300, 3100, 3200, 4000, 4300, 4334, 5000, 1000, 1210, 1300, 1250, 1500, 1450, 1700, 1720, 2150, 2300, 3100, 3200, 4000, 4300, 4334, 5000])
W_more_fields = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0]])
pmw(W_more_fields, x_more_fields) 

T (Threshold) = 1634.9642072606896
query_answers (using pmw): [237115.9385212506, 28546.21498732273, 14719.384904770515, 13742.706126708059, -2722.828151125186]
Update Count = 5


([237115.9385212506,
  28546.21498732273,
  14719.384904770515,
  13742.706126708059,
  -2722.828151125186],
 58279.983390838584)

Conclusion: Adding fields didn't really change much, x still isn't updating.

## 3. Try Dense-Sparse-Dense

### Randomized Permutation

In [21]:
W_test = np.array([[1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 0, 0, 0, 0],
         [0, 1, 0, 1, 0, 0, 0, 0],
         [1, 0, 1, 0, 0, 0, 0, 0], 
         [0, 0, 0, 0, 1, 1, -1, -1]])
np.random.permutation(W_test)

array([[ 0,  1,  0,  1,  0,  0,  0,  0],
       [ 1,  1,  1,  1,  0,  0,  0,  0],
       [ 1,  0,  1,  0,  0,  0,  0,  0],
       [ 1,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  1,  1, -1, -1]])

### Dense -> Sparse

In [22]:
W_allrange = workload.AllRange(8).dense_matrix() #dense
W_identity = workload.Identity(8).dense_matrix() #sparse

In [23]:
W_ds = np.append(W_allrange, W_identity, axis=0) #dense-sparse

In [ ]:
n = 10000
aggregate_avg_diff = 0
for i in range(n):
    (queries, avg_diff) = pmw(W_ds, x_toy)
    aggregate_avg_diff += avg_diff
aggregate_avg_diff = aggregate_avg_diff / n
print(aggregate_avg_diff)

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

...

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Update Count = 0
T (Threshold) = 74642.2219003834
query_answers (using pmw): [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.125, 0.25, 0.375, 0.5, 0.625, 0.125, 0.25, 0.375, 0.5, 0.125, 0.25, 0.375, 0

### Sparse -> Dense 

In [ ]:
W_sd = np.append(W_allrange, W_identity, axis=0) #sparse-dense

In [ ]:
n = 10000
aggregate_avg_diff = 0
for i in range(n):
    (queries, avg_diff) = pmw(W_sd, x_toy) 
    aggregate_avg_diff += avg_diff
aggregate_avg_diff = aggregate_avg_diff / n
print(aggregate_avg_diff)

### Dense -> Sparse -> Dense

### Sparse -> Dense -> Sparse

## 4. Try Non-Smooth Databases

## 5. Assess effects on Analysts